### This script is taking all the chapter abstract text, process it, and then call paraphrasing model to perform text based Augmentation. Then save the result as a image(.png) format. Finally, it goes through all the images and perform image transformation and save the result as .png format.

In [7]:
## importing the modules from ETDAugmentation script
from ipynb.fs.full.ETDaugmentation import *
from imgaug import augmenters as iaa
import imgaug as ia
import pandas as pd
import numpy as np
import cv2
import glob
import re
import time
from tqdm import tqdm
import tensorflow as tf

In [2]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2023-05-05 02:35:56.889855: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 02:35:57.712171: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 8125 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:1d:00.0, compute capability: 7.0


In [5]:
class CHAPTERABS_TEXT_IMG_AUG:
    
    def list_of_phrases(self, phrases_cabs):
        phrases_list = []
        self.phrases_cabs = phrases_cabs
        for text in tqdm(self.phrases_cabs, desc = 'Processing list of phrases'):
            chapter_abs = parser.preprocess_cabs_abs(text)
            chapter_abs_paraphrase = augmentation.paraphrased_text(chapter_abs)
            chapter_abs_list = [chapter_abs_paraphrase]
            phrases_list.append(chapter_abs_list)
        
        return phrases_list

    ## This function is taking all the augmented text and wrapping it so that it can fit into the page
    ## it also adds heading to the top of the page

    def list_of_chapter_abs(self, chapter_abs_text):
        list_chapter_abs = []
        counter = 0
        self.chapter_abs_text = chapter_abs_text
        for text in tqdm(self.chapter_abs_text, desc = 'Processing list of abstract'):
            for row in text:
                chapter_abs_text_wrap = augmentation.wrap_text(row)
                chapter_abs_title = "\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tAbstract\n" + chapter_abs_text_wrap
                list_chapter_abs.append(chapter_abs_title)
        
        return list_chapter_abs
    
    def text_on_img(self, fileName, text, size):
        self.fileName = fileName
        self.text = text
        self.size = size
        self.W = 2360
        self.H = 3200
        font = ImageFont.truetype('NimbusMonoPS-Regular.otf', size)
        image = Image.new(mode = "RGB", size = (self.W, self.H), color = "white")
        draw = ImageDraw.Draw(image)
        w, h = draw.textsize(text)
        ## Adjust the texual position in a page and draw the text on a image
        '''
        dedication -- 14 width and height 8; 
        ack -- 15 width and 6 height; 
        gabs -- 13 width and height 16
        abstract -- 16 width and height 12
        chapter abstract -- 18 width and height 6
        '''
        draw.text(((self.W-w)/18,(self.H-h)/6), text, font=font, fill=(0,0,0), spacing=60)
        image.save(fileName)
        
        return image
    
    ## This function is calling text_on_image function and save the image to the directory
    def chapter_abs_aug(self, chapter_abs_text_title):
        self.chapter_abs_text_title = chapter_abs_text_title
        for n, row in enumerate(self.chapter_abs_text_title):
            fileName = ("/home/mchou001/Label-Abstract/aug_text{}.png".format(n))
            save_image = self.text_on_img(fileName, row, 39) ## dedication -- 43, ack -- 40, chapter abstract - 39 and abstract - 40, gabs -- 38
        
        return save_image
    
    def img_transform(self, img_files):
        self.img_files = img_files
        
        img_augmentation = iaa.Sequential([
            #iaa.Affine(rotate=(-6, 6)),
            #iaa.AdditiveGaussianNoise(scale=(10, 50)),
            iaa.SaltAndPepper(p=0.09),
            #iaa.GaussianBlur(sigma=0.5),
            #iaa.LinearContrast(alpha=1),
            #iaa.PerspectiveTransform(scale=0.025, keep_size=True)
        ], random_order = True)
        
        for n, images in tqdm(enumerate(self.img_files[0:]), desc = 'Processing augmented text to images'):
            aug_image = img_augmentation(image=cv2.imread(images))
            cv2.imwrite('./Label-Abstract/503/aug{}.png'.format(n),aug_image)
        
        # lower = 0
        # upper = 1000
        # while lower < upper:
        #     for n, images in enumerate(self.img_files[0:]):
        #         aug_image = img_augmentation(image=cv2.imread(images))
        #         cv2.imwrite('/Label-Abstract/503/aug{}.png'.format(lower),aug_image)
        #         lower += 1

In [ ]:
if __name__ == "__main__":

    start = time.time()
    with tf.device('/gpu:0'):
    
        parser = Preprocessor()
        augmentation = Augmentation()

        cabs_text = CHAPTERABS_TEXT_IMG_AUG()

        etd_data = pd.read_csv('ETD_aug.csv')
        etd_data.set_index("class", inplace = True)

        label_cabs = etd_data.loc["Label-Abstract"]
        phrases_cabs = label_cabs['text'][0:300]

        cabs = cabs_text.list_of_phrases(phrases_cabs)

        cabs_ = cabs_text.list_of_chapter_abs(cabs)

        df = pd.DataFrame(cabs_, columns = ['text'])
        df.to_csv('./Label-Abstract/abs.csv', encoding = 'utf-8', index= None)

        cabs_text_aug = cabs_text.chapter_abs_aug(cabs_)

        img_path = sorted(glob.glob('./Label-Abstract/*.png'), key = parser.numericalSort)

        augmented_img = cabs_text.img_transform(img_path)
    
    stop = time.time()
    print(f'Generating Images took: {(stop-start)/60} minutes')

Processing list of abstract: 100%|██████████| 300/300 [00:00<00:00, 3014.48it/s]
Processing augmented text to images: 52it [00:37,  1.40it/s]

In [8]:
import os

# folder path
dir_path = r'./Label-Abstract/503/'
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

File count: 300


In [4]:
#!zip -r abs_txt_img.zip ./Label-Abstract/503/

In [9]:
parser = Preprocessor()
image_path = sorted(glob.glob('/home/mchou001/Label-Abstract/503/*.png'), key = parser.numericalSort)

df = pd.DataFrame(image_path, columns = ['img_path'])
df.to_csv('./Label-Abstract/abs_img_aug.csv', encoding = 'utf-8', index= None)

In [10]:
df1 = pd.read_csv('./Label-Abstract/abs_img_aug.csv')
df2 = pd.read_csv('./Label-Abstract/abs.csv')
df1

,img_path
0,/home/mchou001/Label-Abstract/503/aug0.png
1,/home/mchou001/Label-Abstract/503/aug1.png
2,/home/mchou001/Label-Abstract/503/aug2.png
3,/home/mchou001/Label-Abstract/503/aug3.png
4,/home/mchou001/Label-Abstract/503/aug4.png
...,...
295,/home/mchou001/Label-Abstract/503/aug295.png
296,/home/mchou001/Label-Abstract/503/aug296.png
297,/home/mchou001/Label-Abstract/503/aug297.png
298,/home/mchou001/Label-Abstract/503/aug298.png


In [11]:
df3 = [txt.strip('\t') for txt in df2['text']]
df4 = pd.DataFrame(df3, columns = ['text'])
df4['class'] = np.nan
df4 = df4.fillna("Label-Abstract")
df4.to_csv('./Label-Abstract/updated_abs.csv', encoding = 'utf-8', index= None)
df4

,text,class
0,Abstract\nThe Department of Nuclear Engineerin...,Label-Abstract
1,Abstract\nTerry Winograd submitted to the Depa...,Label-Abstract
2,Abstract\nAn empirical investigation of oil sp...,Label-Abstract
3,Abstract\nThe paper was submitted to the Depar...,Label-Abstract
4,Abstract\nThe degree of Doctor of Philosophy w...,Label-Abstract
...,...,...
295,Abstract\nThere are several logical premises t...,Label-Abstract
296,Abstract\nThere are four identifiable approach...,Label-Abstract
297,Abstract\nThere were two stimuli available to ...,Label-Abstract
298,Abstract\nI hypothesis was rejected. The mean ...,Label-Abstract


In [12]:
df5 = pd.read_csv('./Label-Abstract/updated_abs.csv')
df6 = pd.concat([df1, df4], axis = 1)
df6.to_csv('./Label-Abstract/abstract.csv', encoding = 'utf-8', index= None)

In [13]:
df6

,img_path,text,class
0,/home/mchou001/Label-Abstract/503/aug0.png,Abstract\nThe Department of Nuclear Engineerin...,Label-Abstract
1,/home/mchou001/Label-Abstract/503/aug1.png,Abstract\nTerry Winograd submitted to the Depa...,Label-Abstract
2,/home/mchou001/Label-Abstract/503/aug2.png,Abstract\nAn empirical investigation of oil sp...,Label-Abstract
3,/home/mchou001/Label-Abstract/503/aug3.png,Abstract\nThe paper was submitted to the Depar...,Label-Abstract
4,/home/mchou001/Label-Abstract/503/aug4.png,Abstract\nThe degree of Doctor of Philosophy w...,Label-Abstract
...,...,...,...
295,/home/mchou001/Label-Abstract/503/aug295.png,Abstract\nThere are several logical premises t...,Label-Abstract
296,/home/mchou001/Label-Abstract/503/aug296.png,Abstract\nThere are four identifiable approach...,Label-Abstract
297,/home/mchou001/Label-Abstract/503/aug297.png,Abstract\nThere were two stimuli available to ...,Label-Abstract
298,/home/mchou001/Label-Abstract/503/aug298.png,Abstract\nI hypothesis was rejected. The mean ...,Label-Abstract
